# Greek Letters Hedging

The following information about options on Apple stock is provided: $ S = 170 $, $ \tau = T - t = 1 $ month, $ r = 4\% $ per annum (C.C.), and $ \sigma = 35\% $ per year, and consider the following options portfolio:

|Number of contracts (minus sign indicates option written)|
|---------------------------------------------------------|

| Strike price | Calls | Puts |
|--------------|-------|------|
| 165          | 100   | -50  |
| 170          | 150   | -100 |
| 175          | -120  | 0    |

### Tasks
a) Find the net delta ($ \Delta $), gamma ($ \Gamma $), and vega ($ \nu $) exposures of this portfolio.

b) How many shares must be sold/purchased to achieve a delta-neutral ($ \Delta $-neutral) position? What is the market value of those shares?

c) What will happen to the value of the options portfolio if the stock price immediately increases to 175? What is the net gain/loss on the delta-hedged position?

d) Now suppose that the portfolio owner decides to make the portfolio both delta ($ \Delta $) and gamma ($ \Gamma $) neutral by selling/buying shares and modifying the number of 170-strike price call options in the portfolio. How many of these calls should be bought or sold to achieve a gamma-neutral ($ \Gamma $-neutral) position? What is the new value for the number of shares that should be bought or sold for delta-neutrality?

e) What will be the change in the value of the portfolio in (d) if the stock price increases immediately from 170 to 175? Compare this change to your answer in (c) for the portfolio that was only delta-hedged. Briefly comment.

f) Repeat (e) for a stock price decrease from 170 to 165.

                          

## Part a)

We will price the options using the Black-Scholes model and calculate the Greeks (Delta, Gamma, Vega) for each option in the portfolio. The net exposure of the portfolio will be the sum of the individual exposures weighted by the number of contracts for each option.

In [5]:
import pandas as pd
import yfinance as yf
import numpy as np
import warnings
from scipy.stats import norm
warnings.simplefilter(action='ignore', category=FutureWarning)

def download_stock_data(ticker, start_date, end_date):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    return stock_data

def calculate_daily_returns(closing_prices):
    daily_returns = closing_prices.pct_change().dropna()
    return daily_returns

aapl_data = download_stock_data('AAPL', '2021-01-01', '2022-01-01')
tsla_data = download_stock_data('TSLA', '2021-01-01', '2022-01-01')

aapl_returns = calculate_daily_returns(aapl_data['Close'])
tsla_returns = calculate_daily_returns(tsla_data['Close'])

def bs(S, K, tau, r, sigma, q=0):
    """Returns the Black-Scholes parameters d1, d2, N(d1), N(d2), N'(d1)"""
    d1 = (np.log(S / K) + tau * (r - q + 0.5 * sigma ** 2)) / (sigma * np.sqrt(tau))
    d2 = d1 - sigma * np.sqrt(tau)
    N_d1 = norm.cdf(d1)
    N_d2 = norm.cdf(d2)
    N_prime_d1 = (1 / np.sqrt(2 * np.pi)) * np.exp(-0.5 * (d1 ** 2))
    return d1, d2, N_d1, N_d2, N_prime_d1
    
    
def bs_greeks(S, K, tau, r, sigma, q=0, option_type='call'):
    """
    Calculate the Black-Scholes Greeks for a European option.
    :return: Delta, Gamma, Vega
    """
    d1, d2, N_d1, N_d2, N_prime_d1 = bs(S, K, tau, r, sigma, q)
    
    if option_type == 'call':
        delta = np.exp(-q * tau) * N_d1
    else:
        delta = np.exp(-q * tau) * (N_d1 - 1)
        
    gamma = np.exp(-q * tau) * N_prime_d1 / (S * sigma * np.sqrt(tau))
    vega = S * np.exp(-q * tau) * N_prime_d1 * np.sqrt(tau)
    
    return delta, gamma, vega

def bs_price(S, K, tau, r, sigma, q=0, option_type='call'):
    """
    Calculate the Black-Scholes price for a European option.
    """

    d1, d2, N_d1, N_d2, N_prime_d1 = bs(S, K, tau, r, sigma, q)
    
    if option_type == 'call':
        price = S * np.exp(-q * tau) * N_d1 - K * np.exp(-r * tau) * N_d2
    else:
        price = K * np.exp(-r * tau) * (1 - N_d2) - S * np.exp(-q * tau) * (1 - N_d1)
    
    return price


portfolio_a = pd.DataFrame({'Option Type': ['Call', 'Call', 'Call', 'Put', 'Put'], 'Strike Price': [165, 170, 175, 165, 170], 'Number of Contracts': [100, 150, -120, -50, -100]})
S = 170  # Stock price
tau = 1/12  # Time to maturity in years (1 month)
r = 0.04  # Risk-free rate
sigma = 0.35  # Volatility
q = 0  # Dividend yield

print("Stock Price:", S)
print("Time to Maturity (in years):", tau)
print("Risk-Free Rate:", r)
print("Volatility:", sigma)
print('\n')

# Calculate the Greeks for each option in the portfolio
portfolio_a['Delta'], portfolio_a['Gamma'], portfolio_a['Vega'] = zip(*portfolio_a.apply(lambda x: bs_greeks(S, x['Strike Price'], tau, r, sigma, q, x['Option Type'].lower()), axis=1))

# Calculate the price of each option in the portfolio
portfolio_a["Option Price"] = portfolio_a.apply(lambda x: bs_price(S, x['Strike Price'], tau, r, sigma, q, x['Option Type'].lower()), axis=1)


# Calculate greeks of the entire portfolio
net_delta = (portfolio_a['Number of Contracts'] * portfolio_a['Delta']).sum()
net_gamma = (portfolio_a['Number of Contracts'] * portfolio_a['Gamma']).sum()
net_vega = (portfolio_a['Number of Contracts'] * portfolio_a['Vega']).sum()

print("Net Delta Exposure:", round(net_delta, 4))
print("Net Gamma Exposure:", round(net_gamma, 4))
print("Net Vega Exposure:", round(net_vega, 4))
print('\n')

portfolio_a

Stock Price: 170
Time to Maturity (in years): 0.08333333333333333
Risk-Free Rate: 0.04
Volatility: 0.35


Net Delta Exposure: 158.7165
Net Gamma Exposure: -0.492
Net Vega Exposure: -414.7037


,Option Type,Strike Price,Number of Contracts,Delta,Gamma,Vega,Option Price
0,Call,165,100,0.647648,0.021617,18.221363,9.872006
1,Call,170,150,0.533277,0.023146,19.509854,7.124570
2,Call,175,-120,0.419414,0.022751,19.177204,4.951825
3,Put,165,-50,-0.352352,0.021617,18.221363,4.322921
4,Put,170,-100,-0.466723,0.023146,19.509854,6.558847


## Part b)

In [6]:
print("Number of shares to be purchased for delta-neutral position (a negative value indicates selling):", round(-net_delta, 4))
print("Market value of shares for delta-neutral position:", round(net_delta * S, 4)) 

Number of shares to be purchased for delta-neutral position (a negative value indicates selling): -158.7165
Market value of shares for delta-neutral position: 26981.8106


## Part c)

In [7]:
# Calculate the change in the value of the options portfolio if the stock price increases to 175

S = 175  # New stock price

portfolio_a['Option Price New'] = portfolio_a.apply(lambda x: bs_price(S, x['Strike Price'], tau, r, sigma, q, x['Option Type'].lower()), axis=1)
portfolio_a['Option Price Change'] = portfolio_a['Option Price New'] - portfolio_a["Option Price"]
portfolio_a['Total Change in Value'] = portfolio_a['Option Price Change'] * portfolio_a['Number of Contracts']

net_gain_loss = portfolio_a['Total Change in Value'].sum()
print("Change in Value of the Options Portfolio if Stock Price Increases to 175:", round(net_gain_loss, 4))

net_gain_loss_hedged = net_gain_loss - net_delta * (S - 170)
print("Net Gain/Loss on the Delta-Hedged Position:", round(net_gain_loss_hedged, 4))

portfolio_a.round(2)

Change in Value of the Options Portfolio if Stock Price Increases to 175: 786.2933
Net Gain/Loss on the Delta-Hedged Position: -7.2893


,Option Type,Strike Price,Number of Contracts,Delta,Gamma,Vega,Option Price,Option Price New,Option Price Change,Total Change in Value
0,Call,165,100,0.65,0.02,18.22,9.87,13.37,3.49,349.47
1,Call,170,150,0.53,0.02,19.51,7.12,10.07,2.95,442.31
2,Call,175,-120,0.42,0.02,19.18,4.95,7.33,2.38,-285.87
3,Put,165,-50,-0.35,0.02,18.22,4.32,2.82,-1.51,75.27
4,Put,170,-100,-0.47,0.02,19.51,6.56,4.51,-2.05,205.13


## Part d)

In [8]:
# Given data
S = 170  # Stock price
K_170_call_greeks = bs_greeks(S, 170, tau, r, sigma, q, 'call')
K_170_call_gamma = K_170_call_greeks[1]

number_of_calls = -net_gamma / K_170_call_gamma
print("Number of 170-strike price call options to be bought/sold for gamma-neutral position:", round(number_of_calls, 4))

# Calculate the new number of shares for delta-neutrality
net_delta_gamma_neutral = net_delta + number_of_calls * K_170_call_greeks[0]
print("New number of shares for delta-neutrality in gamma-neutral positio (a negative value indicates selling):", round(-net_delta_gamma_neutral, 4))

Number of 170-strike price call options to be bought/sold for gamma-neutral position: 21.2561
New number of shares for delta-neutrality in gamma-neutral positio (a negative value indicates selling): -170.0519


## Part e)

In [9]:
# Calculate the change in the value of the options portfolio if the stock price increases to 175

portfolio_b = pd.DataFrame({'Option Type': ['Call', 'Call', 'Call', 'Put', 'Put'], 'Strike Price': [165, 170, 175, 165, 170], 'Number of Contracts': [100, 150 + number_of_calls, -120, -50, -100]})
portfolio_b['Delta'], portfolio_b['Gamma'], portfolio_b['Vega'] = zip(*portfolio_b.apply(lambda x: bs_greeks(S, x['Strike Price'], tau, r, sigma, q, x['Option Type'].lower()), axis=1))
portfolio_b["Option Price"] = portfolio_b.apply(lambda x: bs_price(S, x['Strike Price'], tau, r, sigma, q, x['Option Type'].lower()), axis=1)

S = 175  # New stock price

portfolio_b['Option Price New'] = portfolio_b.apply(lambda x: bs_price(S, x['Strike Price'], tau, r, sigma, q, x['Option Type'].lower()), axis=1)
portfolio_b['Option Price Change'] = portfolio_b['Option Price New'] - portfolio_b["Option Price"]
portfolio_b['Total Change in Value'] = portfolio_b['Option Price Change'] * portfolio_b['Number of Contracts']

net_gain_loss_delta_gamma_neutral = portfolio_b['Total Change in Value'].sum() - net_delta_gamma_neutral * (S - 170)
print("Change in Value of the Portfolio in Delta-Gamma Neutral Position if Stock Price Increases to 175:", round(net_gain_loss_delta_gamma_neutral, 4))

portfolio_b.round(2)

Change in Value of the Portfolio in Delta-Gamma Neutral Position if Stock Price Increases to 175: -1.2884


,Option Type,Strike Price,Number of Contracts,Delta,Gamma,Vega,Option Price,Option Price New,Option Price Change,Total Change in Value
0,Call,165,100.00,0.65,0.02,18.22,9.87,13.37,3.49,349.47
1,Call,170,171.26,0.53,0.02,19.51,7.12,10.07,2.95,504.98
2,Call,175,-120.00,0.42,0.02,19.18,4.95,7.33,2.38,-285.87
3,Put,165,-50.00,-0.35,0.02,18.22,4.32,2.82,-1.51,75.27
4,Put,170,-100.00,-0.47,0.02,19.51,6.56,4.51,-2.05,205.13


Comparing the answer in part e) for the detla-gamma neutral portfolio to the delta-hedged portfolio in part c), we observe that as the price of the underlying stock increases from 170 to 175, the change in the value of the portfolio is significantly reduced when the portfolio is delta-gamma neutral. The delta-gamma neutral is not only hedge against first order risk (delta) but also against second order risk (gamma). This results in a smaller change in the portfolio value when the stock price moves, indicating a more effective hedging strategy that accounts for both delta and gamma exposures.

## Part f)

In [10]:
# Calculate the change in the value of the options portfolio if the stock price decreases to 165

S = 165
portfolio_b['Option Price New'] = portfolio_b.apply(lambda x: bs_price(S, x['Strike Price'], tau, r, sigma, q, x['Option Type'].lower()), axis=1)
portfolio_b['Option Price Change'] = portfolio_b['Option Price New'] - portfolio_b["Option Price"]
portfolio_b['Total Change in Value'] = portfolio_b['Option Price Change'] * portfolio_b['Number of Contracts']

net_gain_loss_delta_gamma_neutral = portfolio_b['Total Change in Value'].sum() - net_delta_gamma_neutral * (S - 170)
print("Change in Value of the Portfolio in Delta-Gamma Neutral Position if Stock Price Decreases to 165:", round(net_gain_loss_delta_gamma_neutral, 4))

portfolio_b.round(2)

Change in Value of the Portfolio in Delta-Gamma Neutral Position if Stock Price Decreases to 165: 1.3552


,Option Type,Strike Price,Number of Contracts,Delta,Gamma,Vega,Option Price,Option Price New,Option Price Change,Total Change in Value
0,Call,165,100.00,0.65,0.02,18.22,9.87,6.92,-2.96,-295.70
1,Call,170,171.26,0.53,0.02,19.51,7.12,4.75,-2.37,-406.55
2,Call,175,-120.00,0.42,0.02,19.18,4.95,3.13,-1.82,218.10
3,Put,165,-50.00,-0.35,0.02,18.22,4.32,6.37,2.04,-102.15
4,Put,170,-100.00,-0.47,0.02,19.51,6.56,9.18,2.63,-262.61


Comparing part f) to part c), we can draw the same conclusion as in part e): Since the delta-gamma neutral portfolio is hedged against both first-order (delta) and second-order (gamma) risks, the change in the portfolio value is significantly reduced when the stock price decreases from 170 to 165. This demonstrates the effectiveness of a delta-gamma neutral strategy in mitigating the impact of stock price movements on the options portfolio.

# Question-3 (VaR-1): 

This question is intended to help you understand how to estimate Value-at-Risk (VaR) and Conditional Value-at-Risk (CVaR). In parts (a) through (e), please use the variance-covariance approach. In parts (f), you need to use the historical simulation approach. You will need to use your Apple/Tesla dataset of the prior question.

a) Using the variance-covariance approach, estimate the one-day VaR at the 99% confidence level for a \\$1,000,000 investment in Apple. What would be the one-day VaR for a \\$1,000,000 investment in Tesla?
b) Compute the correlation coefficient between the returns on Apple and Tesla.
c) Using your estimates in parts (a) and (b), find the standard deviation of an equally-weighted portfolio that invests in Apple and Tesla.
d) Using the variance-covariance approach, estimate the one-day VaR at the 99% confidence level for a \\$1,000,000 investment in the equally-weighted portfolio.
e) Repeat part (d) using the historical simulation approach. You may use the "SORT" function under the DATA menu in Excel.
f) Using the historical simulation approach, estimate: (i) the one-day CVaR for a \\$1,000,000 investment in Apple; (ii) the one-day CVaR for a \\$1,000,000 investment in Tesla; and (iii) the one-day CVaR for a \\$1,000,000 investment in the equally-weighted portfolio.

## Part a)

In [11]:
# Calculate the one-day VaR at the 99% confidence level for a $1,000,000 investment in Apple and Tesla using the variance-covariance approach

investment = 1000000  # Investment amount
confidence_level = 0.99  # Confidence level

# Calculate the mean and standard deviation of daily returns for Apple and Tesla
aapl_mean_return = aapl_returns.mean()
aapl_std_return = aapl_returns.std()

tsla_mean_return = tsla_returns.mean()
tsla_std_return = tsla_returns.std()

appl_std_inv = investment * aapl_std_return
tsla_std_inv = investment * tsla_std_return

# Calculate the VaR for Apple and Tesla
appl_1day_VaR = norm.ppf(confidence_level) * appl_std_inv
tsla_1day_VaR = norm.ppf(confidence_level) * tsla_std_inv

print("One-day VaR for a $1,000,000 investment in Apple:", round(appl_1day_VaR, 2))
print("One-day VaR for a $1,000,000 investment in Tesla:", round(tsla_1day_VaR, 2))

One-day VaR for a $1,000,000 investment in Apple: 36665.51
One-day VaR for a $1,000,000 investment in Tesla: 80267.61


## Part b)

In [12]:
# Computing the correlation coefficient between the returns on Apple and Tesla

correlation_coefficient = aapl_returns.corr(tsla_returns)
print("Correlation Coefficient between Apple and Tesla Returns:", round(correlation_coefficient, 4))

Correlation Coefficient between Apple and Tesla Returns: 0.458


## Part c)

In [13]:
# Using the estimates from parts (a) and (b) to find the standard deviation of an equally-weighted portfolio

# Standard deviation of an equally-weighted portfolio
portfolio_std = np.sqrt(0.5**2 * aapl_std_return**2 + 0.5**2 * tsla_std_return**2 + 2 * 0.5 * 0.5 * correlation_coefficient * aapl_std_return * tsla_std_return)
print("Standard Deviation of an Equally-Weighted Portfolio:", round(portfolio_std, 4))

Standard Deviation of an Equally-Weighted Portfolio: 0.022


## Part d)

In [14]:
# Estimate the one-day VaR at the 99% confidence level for a $1,000,000 investment in the equally-weighted portfolio using the variance-covariance approach

portfolio_1day_VaR = norm.ppf(confidence_level) * investment * portfolio_std
print("One-day VaR for a $1,000,000 investment in the equally-weighted portfolio:", round(portfolio_1day_VaR, 2))

One-day VaR for a $1,000,000 investment in the equally-weighted portfolio: 51193.47


## Part e)

In [15]:
# Estimate the one-day VaR at the 99% confidence level for a $1,000,000 investment in the equally-weighted portfolio using the historical simulation approach

# Calculate the returns of the equally-weighted portfolio
portfolio_returns = 0.5 * aapl_returns + 0.5 * tsla_returns

# Calculate the historical VaR
historical_VaR = -np.percentile(portfolio_returns, 1, method='lower') * investment

print("One-day VaR for a $1,000,000 investment in the equally-weighted portfolio using historical simulation:", round(historical_VaR, 2))

One-day VaR for a $1,000,000 investment in the equally-weighted portfolio using historical simulation: 57648.73


## Part f)

In [16]:
def calculate_cvar(returns, investment_amount, confidence_level=0.99):
    var_threshold = np.percentile(returns, 100 * (1 - confidence_level), method='lower')
    cvar = returns[returns <= var_threshold].mean() * investment_amount
    return -cvar

investment = 1000000  # $1,000,000 investment
confidence_level = 0.99  # 99% confidence level
aapl_cvar = calculate_cvar(aapl_returns, investment, confidence_level)
tsla_cvar = calculate_cvar(tsla_returns, investment, confidence_level)
portfolio_returns = 0.5 * aapl_returns + 0.5 * tsla_returns
portfolio_cvar = calculate_cvar(portfolio_returns, investment, confidence_level)

print(f"One-day CVaR for a $1,000,000 investment in Apple: ${aapl_cvar:,.2f}")
print(f"One-day CVaR for a $1,000,000 investment in Tesla: ${tsla_cvar:,.2f}")
print(f"One-day CVaR for a $1,000,000 investment in the equally-weighted portfolio: ${portfolio_cvar:,.2f}")

One-day CVaR for a $1,000,000 investment in Apple: $39,452.65
One-day CVaR for a $1,000,000 investment in Tesla: $95,330.77
One-day CVaR for a $1,000,000 investment in the equally-weighted portfolio: $58,019.08
